<a href="https://colab.research.google.com/github/Rama-Has/Poem_Generator_Rama_Hasiba_12010022/blob/main/Arabic_Poem_Generator_NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers==4.28.0
!pip install --upgrade accelerate
!pip install git+https://github.com/huggingface/accelerate
# !pip install transformers==4.28.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00


In [8]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 4.7 MB/s eta 0:00:00


In [51]:
!pip install arabic_reshaper
!pip install python-bidi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


--------------------------------------------
> # Arabic Poem Generator  




In [52]:
from google.colab import drive
import numpy as np
from google.colab import files
import shutil
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer \
,TextDataset, DataCollatorForLanguageModeling, pipeline
from sklearn.model_selection import train_test_split
from pyarabic.araby import strip_tashkeel 
from google.colab import drive
import tensorflow as tf
import pandas as pd  
import re 
from wordcloud import WordCloud, STOPWORDS
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib.pyplot as plt

# Reading Data

In [10]:
#mount google drive to the colab runtime
drive.mount('/content/drive', force_remount = True) 

Mounted at /content/drive


In [11]:
!unzip -q "/content/drive/MyDrive/Arabic Poem Comprehensive Dataset (APCD).zip" -d "/content/"

In [13]:
data = pd.read_csv("/content/Arabic Poem Comprehensive Dataset (APCD).csv")
data

,العصر,الشاعر,الديوان,القافية,البحر,الشطر الايسر,الشطر الايمن,البيت
0,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن...
1,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَلا سُرعَتي يَوماً بِسابِقَةِ الرَدى,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُ...
2,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَ...
3,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,تُؤامِرُني سِرّاً لِأَصرِمَ مَرثَدا,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُ...
4,قبل الإسلام,عمرو بنِ قُمَيئَة,الديوان الرئيسي,د,الطويل,وَأَفرَعَ في لَومي مِراراً وَأَصعَدا,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفر...
...,...,...,...,...,...,...,...,...
1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تَتَغنى,هي أغلى ما أنشأ اللَّه في الدنيا,هي أغلى ما أنشأ اللَّه في الدنيا وأحلى قصيد...
1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجداً ويخفق حسنا,هي شلال بهجة وبهاء,هي شلال بهجة وبهاء يتداعى وجداً ويخفق حسنا
1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجناً فسجنا,هي حلم الهوى ومنطلقي الباقي,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


# General Look at the data

In [15]:
print(f"There are a atotal of {data.shape[0]} poem line in the dataset, I'll use the first 50 000 sample  and take البيت column.")

There are a atotal of 1831770 poem line in the dataset, I'll use the first 12 000 sample  and take البيت column.


In [16]:
data = data[['البيت']]
data

,البيت
0,خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا وَأَن...
1,فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ وَلا سُ...
2,وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً وَتَ...
3,لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ تُؤامِرُ...
4,وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ وَأَفر...
...,...
1831765,هي أغلى ما أنشأ اللَّه في الدنيا وأحلى قصيد...
1831766,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1831767,هي شلال بهجة وبهاء يتداعى وجداً ويخفق حسنا
1831768,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


In [17]:
poems_lines = data['البيت'].to_list()[: 50000]
poems_lines[0: 5]

['خَليلَيَّ لا تَستَعجِلا أَن تَزَوَّدا    وَأَن تَجمَعا شَملي وَتَنتَظِرا غَدا',
 'فَما لَبَثٌ يَوماً بِسابِقٍ مَغنَمٍ    وَلا سُرعَتي يَوماً بِسابِقَةِ الرَدى',
 'وَإِن تُنظِراني اليَومَ أَقضِ لُبانَةً    وَتَستَوجِبا مَنّاً عَلَيَّ وَتُحمَدا',
 'لَعَمرُكَ ما نَفسٌ بِجِدٍ رَشيدَةٍ    تُؤامِرُني سِرّاً لِأَصرِمَ مَرثَدا',
 'وَإِن ظَهَرَت مِنهُ قَوارِصُ جَمَّةٌ    وَأَفرَعَ في لَومي مِراراً وَأَصعَدا']

## Data Cleanning:
  1. Remove strips.
  2. Remove arabic punctuation.
  3. Remove multiple spaces.

In [19]:
# remove special characters
ARABIC_PUNCTUATION = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

def remove_arabic_punctuation(text: str):
    translator = str.maketrans('', '', ARABIC_PUNCTUATION)
    cleaned_text = text.translate(translator)
    return cleaned_text

def remove_multiple_spaces(text):
    cleaned_text = re.sub(' +', ' ', text)
    return cleaned_text

In [26]:
poems_lines_df = data[['البيت']].head(50000).copy()
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_arabic_punctuation)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(strip_tashkeel)
poems_lines_df['البيت'] = poems_lines_df['البيت'].map(remove_multiple_spaces)

In [56]:
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
from PIL import ImageFont
font = ImageFont.load_default()

In [63]:
# # Read poems
# text = " ".join(poems_lines_df['البيت'].to_list())

# # Reshape the text to join the Arabic characters correctly
# reshaped_text = arabic_reshaper.reshape(text)

# # Use the bidi algorithm to handle the right-to-left direction of Arabic text
# bidi_text = get_display(reshaped_text)

# # font = ImageFont.load_default()

# # Create the WordCloud object and generate the word cloud image
# wordcloud = WordCloud(
#     width=800, 
#     height=800, 
#     background_color='white', 
#     font_path= 'arial',#'arial.ttf',
#     min_font_size=10, 
#     max_words=200
#     ).generate(bidi_text)

# # Display the word cloud image
# plt.figure(figsize=(8, 8), facecolor=None)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.tight_layout(pad=0)
# plt.show()

# Finetune GPT-2

In [30]:
poems_lines_df.to_csv(
    r'arabic_full_text.txt', 
    header = None, 
    index = None, 
    sep=' ', 
    mode='a'
    )

In [31]:
file_path = '/content/arabic_full_text.txt'

Splitting the data into train, and test without shuffeling.

In [32]:
train, test = train_test_split(poems_lines_df.head(50000), test_size=0.15, shuffle=False)

In [33]:
train.shape

(42500, 1)

In [34]:
train.columns

Index(['البيت'], dtype='object')

In [35]:
test.shape

(7500, 1)

Write train and test data each one into distinct text file.

In [36]:
train.to_csv(
    r'train_dataset.txt', 
    header=None, 
    index=None, 
    sep=' ', 
    mode='a'
    )

test.to_csv(
    r'test_dataset.txt', 
    header=None, 
    index=None, 
    sep=' ', 
    mode='a'
    )

In [37]:
train.head(3)

,البيت
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وتنتظ...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بسابقة ...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا علي و...


Load the tokenizer that is created specialy to the "aragpt-2base model".

In [38]:
#Load the tokenizer 
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

train_path = '/content/train_dataset.txt'
test_path = '/content/test_dataset.txt'

In [40]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )

    return train_dataset, test_dataset, data_collator

In [41]:
def remove_double_qoutes(file_path: str) -> None: 
  
    # Read the file
    with open(file_path, 'r') as file:
        text = file.read()

    # Remove double quotes
    text_without_quotes = text.replace('"', '')

    # Write the modified text back to the file
    with open(file_path, 'w') as file:
        file.write(text_without_quotes)

remove_double_qoutes(train_path)
remove_double_qoutes(test_path)

In [ ]:
train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

In [43]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")

training_args = TrainingArguments(
    output_dir = "./gpt2-araPoems", #The output directory
    overwrite_output_dir = True, #overwrite the content of the output directory
    num_train_epochs = 15, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [44]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,7.488000
1000,6.058400
1500,5.741200
2000,5.571000


TrainOutput(global_step=2370, training_loss=6.102360026041667, metrics={'train_runtime': 2746.3131, 'train_samples_per_second': 27.511, 'train_steps_per_second': 0.863, 'total_flos': 4935479869440000.0, 'train_loss': 6.102360026041667, 'epoch': 15.0})

In [45]:
trainer.save_model('./aragpt_final_one') 

# Compress the folder into a zip file
shutil.make_archive("/content/aragpt_final_one", "zip", "/content/aragpt_final_one")


'/content/aragpt_final_one.zip'

In [47]:
generator_ = pipeline('text-generation', model='./aragpt_final_one', tokenizer='aubmindlab/aragpt2-base')

In [48]:
max_words = 50  # Set the desired maximum number of words

result = generator_('الليل ', max_length = 60)[0]['generated_text']

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [49]:
print(result)

الليل مذموم
ولقد علمتم أني إذا ما شربت من الخمر ألذ وألذ وأغلى
فإذا أنا لم أشربها فإنني سأشربها وإن كنت لا أشربها
ألم تر أن الدهر يخامره في كل يوم وليلة وفي كل يوم ويوم



## Evaluation

In [66]:
trainer.evaluate(
    test_dataset,
    metric_key_prefix = "eval_perplexity"
)

{'eval_perplexity_loss': 5.859355926513672,
 'eval_perplexity_runtime': 9.0871,
 'eval_perplexity_samples_per_second': 90.898,
 'eval_perplexity_steps_per_second': 1.431,
 'epoch': 15.0}

In [67]:
trainer.evaluate(
    test_dataset,
    metric_key_prefix = "eval_bleu"
)

{'eval_bleu_loss': 5.859355926513672,
 'eval_bleu_runtime': 9.221,
 'eval_bleu_samples_per_second': 89.578,
 'eval_bleu_steps_per_second': 1.41,
 'epoch': 15.0}